In [3]:
from numpy import *
import pandas as pd

from bs4 import BeautifulSoup
from operator import itemgetter
import requests

In [11]:
link_base = 'https://www.netpincer.hu/'

#Parse netpincer to get list of html links for restaurants that deliver to CEU
response = requests.get('https://www.netpincer.hu/restaurants/new?lat=47.501185&lng=19.049364&vertical=restaurants')
soup = BeautifulSoup(response.content, "html.parser") 
#Find all restaurant link in html
restaurants = soup.find('ul', {'class':'vendor-list'}).findAll('a')
#create list of links to iterate over and parse them 
link_list = [link_base+link.get('href') for link in restaurants]

In [266]:
final_df = pd.DataFrame(columns = ['Restaurant', 'Product_Name', 'Price']) 

In [288]:
def netpincer_parser(URL): 
    global final_df
    prices = []
    
    response = requests.get(URL)
    soup = BeautifulSoup(response.content, "html.parser") 

    #Getting name of the restaurant
    restaurant = soup.find('div',{'class':'vendor-info-main-headline item'}).getText()

    #Getting name of product and prices - fortunately both are spans
    smaller_soup = soup.find('div', {'class':'menu__items'}).findAll('span')
    temp_list= [elem.getText() for elem in smaller_soup]

    #Name will be every first, price will be every second element in smaller soup
    product_names = temp_list[::2]
    prices_temp = temp_list[1::2]
    restaurant_name = [restaurant for i in range(len(product_names))]

    #Price requires further alterations
    for elem in prices_temp:
        temp_elem = elem.split()
        prices.append("".join(temp_elem))

    #Put vectors into pandas dataframe    
    dict_for_df = {'Restaurant': restaurant_name, 'Product_Name': product_names, 'Price': prices}  
    df = pd.DataFrame(dict_for_df)
    
    final_df = final_df.append(df)

I have an issue below - will need to debug my code for restaurants where it failed. I was able to get at least 73 restaurants tough. 

In [301]:
final_df = pd.DataFrame(columns = ['Restaurant', 'Product_Name', 'Price']) 

for link in link_list:
    netpincer_parser(link)

ValueError: arrays must all be same length

In [302]:
final_df.describe()

,Restaurant,Product_Name,Price
count,8536,8536,8536
unique,73,7436,924
top,S.O.S. Night - Italházhozszállítás\n,Ketchup,innen2190Ft
freq,963,19,177


In [306]:
final_df.to_csv(r'C:\Users\T450s\Python_directory\trial_netpincer.csv', 
                           index = True, sep=',', encoding='utf-8')